In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np


In [3]:
data=pd.read_csv("/Users/zeelpatel/Desktop/Ineuron/Restaurant_Rating_prediction/notebook/data/cleaned_zomato_data.csv")

In [4]:
data.head()

,Unnamed: 0,online_order,book_table,rate,votes,location,restrofacility,cuisines,cost for two,restro type,outlet location
0,0,1,1,4.1,775,other,Casual Dining,other,800,Buffet,Banashankari
1,1,1,0,4.1,787,other,Casual Dining,other,800,Buffet,Banashankari
2,2,1,0,3.8,918,other,other,other,800,Buffet,Banashankari
3,3,0,0,3.7,88,other,Quick Bites,"South Indian, North Indian",300,Buffet,Banashankari
4,4,0,0,3.8,166,other,Casual Dining,other,600,Buffet,Banashankari


In [5]:
data.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [6]:
data.shape

(49126, 10)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49126 entries, 0 to 49125
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   online_order     49126 non-null  int64  
 1   book_table       49126 non-null  int64  
 2   rate             49126 non-null  float64
 3   votes            49126 non-null  int64  
 4   location         49126 non-null  object 
 5   restrofacility   49126 non-null  object 
 6   cuisines         49126 non-null  object 
 7   cost for two     49126 non-null  int64  
 8   restro type      49126 non-null  object 
 9   outlet location  49126 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 3.7+ MB


In [8]:
data.rename(columns={'restro type':'restro_type','cost for two':'cost_for_two','outlet location':'outlet_location'},inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [10]:
data.location=encoder.fit_transform(data.location)
data.restrofacility=encoder.fit_transform(data.restrofacility)
data.cuisines=encoder.fit_transform(data.cuisines)
data.restro_type=encoder.fit_transform(data.restro_type)
data.outlet_location=encoder.fit_transform(data.outlet_location)

In [11]:
data.head()

,online_order,book_table,rate,votes,location,restrofacility,cuisines,cost_for_two,restro_type,outlet_location
0,1,1,4.1,775,18,2,19,800,0,1
1,1,0,4.1,787,18,2,19,800,0,1
2,1,0,3.8,918,18,8,19,800,0,1
3,0,0,3.7,88,18,6,17,300,0,1
4,0,0,3.8,166,18,2,19,600,0,1


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49126 entries, 0 to 49125
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   online_order     49126 non-null  int64  
 1   book_table       49126 non-null  int64  
 2   rate             49126 non-null  float64
 3   votes            49126 non-null  int64  
 4   location         49126 non-null  int64  
 5   restrofacility   49126 non-null  int64  
 6   cuisines         49126 non-null  int64  
 7   cost_for_two     49126 non-null  int64  
 8   restro_type      49126 non-null  int64  
 9   outlet_location  49126 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 3.7 MB


In [13]:
#splitting the data into X,y
X=data.drop(columns='rate')
y=data['rate']

In [14]:
X.shape,y.shape

((49126, 9), (49126,))

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error

In [16]:
#splitting data into train and test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [17]:
X_train.shape,X_test.shape

((39300, 9), (9826, 9))

In [18]:
lr_model=LinearRegression()
lr_model.fit(X_train,y_train)

LinearRegression()

In [19]:
y_pred=lr_model.predict(X_test)
score=r2_score(y_pred,y_test)
score
mse=mean_absolute_error(y_pred,y_test)
mse

0.7923052453971325

In [20]:
from sklearn.ensemble import RandomForestRegressor
RFR=RandomForestRegressor()
RFR.fit(X_train,y_train)

RandomForestRegressor()

In [21]:

y_pred=RFR.predict(X_test)
score_RFR=r2_score(y_pred,y_test)
score_RFR

0.798738315070245

In [22]:
y_pred_train=RFR.predict(X_train)
score_RFR_train=r2_score(y_pred_train,y_train)
score_RFR_train

0.9595765668252147

In [23]:
params={'n_estimators':[10,20,30,40,50],
        'max_depth':[1,2,3,4,5,6,7],
        'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_CV=GridSearchCV(RFR,param_grid=params,scoring='r2' ,cv=5)

grid_CV.fit(X_train,y_train)


In [1]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error


In [23]:
# Define the number of folds for cross-validation
num_folds = 5

# Create a K-Fold cross-validation object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
mse_scores = cross_val_score(RFR, X, y, cv=kf, scoring='neg_mean_squared_error')

# Convert MSE scores to positive values and calculate RMSE (Root Mean Squared Error)
rmse_scores = np.sqrt(-mse_scores)

# Print RMSE scores for each fold
for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold + 1}: RMSE = {rmse:.3f}")

# Calculate the average RMSE
average_rmse = np.mean(rmse_scores)
print(f"Average RMSE across {num_folds} folds: {average_rmse:.3f}")

Fold 1: RMSE = 0.535
Fold 2: RMSE = 0.525
Fold 3: RMSE = 0.526
Fold 4: RMSE = 0.527
Fold 5: RMSE = 0.507
Average RMSE across 5 folds: 0.524


In [24]:
from sklearn.metrics import make_scorer, r2_score

In [25]:
# Define a custom scoring function for R2
custom_r2_scorer = make_scorer(r2_score)

# Perform cross-validation to calculate R2 scores
cv_scores = cross_val_score(RFR, X, y, cv=5, scoring=custom_r2_scorer)

# Print R2 scores for each fold
for fold, score in enumerate(cv_scores):
    print(f"Fold {fold + 1}: R2 = {score:.3f}")

# Calculate the average R2 score
average_r2 = np.mean(cv_scores)
print(f"Average R2 across folds: {average_r2:.3f}")

Fold 1: R2 = 0.780
Fold 2: R2 = 0.791
Fold 3: R2 = 0.812
Fold 4: R2 = 0.889
Fold 5: R2 = 0.847
Average R2 across folds: 0.824
